In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_12782/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self, metrics1, metrics2):
        r2_val, r2_test = self.results_df[metrics1], self.results_df[metrics2]
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation(metrics1="r2", metrics2="r2_vt")
        self.discard_high_standard_deviation(metrics1="r2_val", metrics2="r2_test")
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_7_9_6,0.973192,0.910181,0.993577,0.946703,0.005015,0.026922,0.001605,0.015008,"Hidden Size=[45, 22], regularizer=0.01, learni..."
1,model_7_9_5,0.973179,0.911618,0.994611,0.947945,0.005018,0.026491,0.001347,0.014658,"Hidden Size=[45, 22], regularizer=0.01, learni..."
2,model_7_9_7,0.973122,0.908871,0.992520,0.945523,0.005028,0.027314,0.001869,0.015340,"Hidden Size=[45, 22], regularizer=0.01, learni..."
3,model_7_9_4,0.973059,0.913218,0.995585,0.949252,0.005040,0.026012,0.001104,0.014290,"Hidden Size=[45, 22], regularizer=0.01, learni..."
4,model_7_9_8,0.972987,0.907665,0.991457,0.944399,0.005054,0.027676,0.002135,0.015657,"Hidden Size=[45, 22], regularizer=0.01, learni..."
5,model_7_9_9,0.972807,0.906542,0.990424,0.943335,0.005087,0.028013,0.002393,0.015956,"Hidden Size=[45, 22], regularizer=0.01, learni..."
6,model_7_9_3,0.972788,0.914964,0.996437,0.950593,0.005091,0.025488,0.000890,0.013913,"Hidden Size=[45, 22], regularizer=0.01, learni..."
7,model_7_9_10,0.972593,0.905486,0.989428,0.942324,0.005127,0.028329,0.002642,0.016241,"Hidden Size=[45, 22], regularizer=0.01, learni..."
8,model_7_9_11,0.972359,0.904505,0.988475,0.941373,0.005171,0.028623,0.002880,0.016509,"Hidden Size=[45, 22], regularizer=0.01, learni..."
9,model_7_9_2,0.972298,0.916813,0.997085,0.951905,0.005182,0.024934,0.000729,0.013543,"Hidden Size=[45, 22], regularizer=0.01, learni..."
